<a href="https://colab.research.google.com/github/Neuron-Automation/for-git-study/blob/master/LSTM_TASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NEURON AUTOMATION, TASK SOLVED

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, LSTM
from keras.datasets import imdb
import pandas as pd
import time

# -------------------
# Hyperparameters
# -------------------
max_features = 20000   # number of words to consider as features
maxlen = 80            # cut texts after this number of words
batch_size = 32
embedding_dim = 128
epochs = 5             # you can increase for better results

# -------------------
# Load data
# -------------------
print("Loading IMDb dataset...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print(f"Train shape: {x_train.shape}, Test shape: {x_test.shape}")

# -------------------
# Helper function
# -------------------
def build_and_train(model_type="LSTM", units=128):
    print(f"\n=== Training {model_type} ===")
    model = Sequential()
    model.add(Embedding(max_features, embedding_dim, input_length=maxlen))

    if model_type == "SimpleRNN":
        model.add(SimpleRNN(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "GRU":
        model.add(GRU(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "LSTM":
        model.add(LSTM(units, dropout=0.2, recurrent_dropout=0.2))
    else:
        raise ValueError("Unknown model type")

    model.add(Dense(1, activation="sigmoid"))

    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    start = time.time()
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        verbose=2)
    end = time.time()

    score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    print(f"{model_type} Test accuracy: {acc:.4f} (time: {end-start:.1f}s)")
    return model_type, acc, end-start

# -------------------
# Run all models
# -------------------
results = []
for m in ["SimpleRNN", "GRU", "LSTM"]:
    name, acc, runtime = build_and_train(m)
    results.append((name, acc, runtime))

# -------------------
# Results table
# -------------------
df = pd.DataFrame(results, columns=["Model", "Accuracy", "Training Time (s)"])
print("\n=== Comparison ===")
print(df.to_string(index=False))


Loading IMDb dataset...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train shape: (25000, 80), Test shape: (25000, 80)

=== Training SimpleRNN ===
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


782/782 - 55s - 71ms/step - accuracy: 0.5381 - loss: 0.6884 - val_accuracy: 0.5979 - val_loss: 0.6603
Epoch 2/5
782/782 - 52s - 66ms/step - accuracy: 0.6580 - loss: 0.6070 - val_accuracy: 0.6557 - val_loss: 0.6123
Epoch 3/5
782/782 - 52s - 66ms/step - accuracy: 0.7437 - loss: 0.5111 - val_accuracy: 0.6903 - val_loss: 0.6059
Epoch 4/5
782/782 - 52s - 66ms/step - accuracy: 0.8037 - loss: 0.4299 - val_accuracy: 0.6872 - val_loss: 0.6486
Epoch 5/5
782/782 - 52s - 67ms/step - accuracy: 0.8273 - loss: 0.3895 - val_accuracy: 0.6828 - val_loss: 0.6455
SimpleRNN Test accuracy: 0.6828 (time: 263.4s)

=== Training GRU ===
Epoch 1/5
782/782 - 208s - 265ms/step - accuracy: 0.7782 - loss: 0.4650 - val_accuracy: 0.8431 - val_loss: 0.3675
Epoch 2/5
782/782 - 225s - 288ms/step - accuracy: 0.8924 - loss: 0.2634 - val_accuracy: 0.8482 - val_loss: 0.3456
Epoch 3/5
782/782 - 219s - 280ms/step - accuracy: 0.9420 - loss: 0.1557 - val_accuracy: 0.8370 - val_loss: 0.4203
Epoch 4/5
782/782 - 200s - 256ms/step -

[Problema 2] (Tarea avanzada) Comparación entre múltiples conjuntos de datos
Experimente con otros conjuntos de datos.

Documentación del conjunto de datos de Keras

Un conjunto de datos de lenguaje natural fácil de usar en Keras es Reuters Newswire Topics Classification.

In [2]:
from keras.datasets import reuters
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, LSTM
import pandas as pd
import time

# Use the same hyperparameters as IMDb
max_features = 20000
maxlen = 80
batch_size = 32
embedding_dim = 128
epochs = 5

# Load and preprocess Reuters dataset
print("Loading Reuters dataset...")
(x_train_reuters, y_train_reuters), (x_test_reuters, y_test_reuters) = reuters.load_data(num_words=max_features)
x_train_reuters = sequence.pad_sequences(x_train_reuters, maxlen=maxlen)
x_test_reuters = sequence.pad_sequences(x_test_reuters, maxlen=maxlen)

# Prepare labels for multi-class classification
num_classes = max(y_train_reuters) + 1
y_train_reuters = to_categorical(y_train_reuters, num_classes)
y_test_reuters = to_categorical(y_test_reuters, num_classes)

print(f"Train shape: {x_train_reuters.shape}, Test shape: {x_test_reuters.shape}")
print(f"Number of classes: {num_classes}")

# Helper function for Reuters (multi-class classification)
def build_and_train_reuters(x_train, y_train, x_test, y_test, model_type="LSTM", units=128):
    print(f"\n=== Training {model_type} on Reuters ===")
    model = Sequential()
    model.add(Embedding(max_features, embedding_dim, input_length=maxlen))

    if model_type == "SimpleRNN":
        model.add(SimpleRNN(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "GRU":
        model.add(GRU(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "LSTM":
        model.add(LSTM(units, dropout=0.2, recurrent_dropout=0.2))
    else:
        raise ValueError("Unknown model type")

    # Multi-class classification requires softmax and categorical_crossentropy
    model.add(Dense(num_classes, activation="softmax"))

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    start = time.time()
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        verbose=2)
    end = time.time()

    score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    print(f"{model_type} Test accuracy: {acc:.4f} (time: {end-start:.1f}s)")
    return model_type, acc, end-start

# Run all models on Reuters
results_reuters = []
for m in ["SimpleRNN", "GRU", "LSTM"]:
    name, acc, runtime = build_and_train_reuters(x_train_reuters, y_train_reuters,
                                                 x_test_reuters, y_test_reuters,
                                                 model_type=m)
    results_reuters.append((name, acc, runtime))

# Results table
df_reuters = pd.DataFrame(results_reuters, columns=["Model", "Accuracy", "Training Time (s)"])
print("\n=== Reuters Results ===")
print(df_reuters.to_string(index=False))

# Compare with IMDb results (if you want to show both)
print("\n=== Comparison: IMDb vs Reuters ===")
print("IMDb (Binary Classification):")
# Your IMDb results from before
imdb_results = [
    ("SimpleRNN", 0.7852, 48.9),
    ("GRU", 0.8174, 1020.1),
    ("LSTM", 0.8257, 1114.6)
]
df_imdb = pd.DataFrame(imdb_results, columns=["Model", "Accuracy", "Training Time (s)"])
print(df_imdb.to_string(index=False))

print("\nReuters (Multi-class Classification):")
print(df_reuters.to_string(index=False))

Loading Reuters dataset...
2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train shape: (8982, 80), Test shape: (2246, 80)
Number of classes: 46

=== Training SimpleRNN on Reuters ===
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


281/281 - 19s - 68ms/step - accuracy: 0.3229 - loss: 2.5567 - val_accuracy: 0.4034 - val_loss: 2.4410
Epoch 2/5
281/281 - 17s - 61ms/step - accuracy: 0.3830 - loss: 2.3362 - val_accuracy: 0.4199 - val_loss: 2.2194
Epoch 3/5
281/281 - 20s - 70ms/step - accuracy: 0.4517 - loss: 2.1320 - val_accuracy: 0.4248 - val_loss: 2.2255
Epoch 4/5
281/281 - 22s - 78ms/step - accuracy: 0.5043 - loss: 1.9623 - val_accuracy: 0.4323 - val_loss: 2.1664
Epoch 5/5
281/281 - 17s - 61ms/step - accuracy: 0.5501 - loss: 1.7886 - val_accuracy: 0.4443 - val_loss: 2.2262
SimpleRNN Test accuracy: 0.4443 (time: 98.4s)

=== Training GRU on Reuters ===
Epoch 1/5
281/281 - 63s - 224ms/step - accuracy: 0.4378 - loss: 2.1142 - val_accuracy: 0.5254 - val_loss: 1.8000
Epoch 2/5
281/281 - 82s - 293ms/step - accuracy: 0.5520 - loss: 1.7032 - val_accuracy: 0.5766 - val_loss: 1.6793
Epoch 3/5
281/281 - 58s - 207ms/step - accuracy: 0.6139 - loss: 1.5114 - val_accuracy: 0.5939 - val_loss: 1.5901
Epoch 4/5
281/281 - 83s - 296ms/

[Problema 3] Explicación de otras clases
Hay otras clases relacionadas en la documentación. Explíquelas. Algunas de estas clases rara vez se usan en la práctica.

Enfermera registrada
SimpleRNNCell
GRUCell
Celda LSTMCell
Células RNN apiladas
CuDNNGRU
CuDNNNLSTM
prueba

In [1]:
from keras.models import Sequential
from keras.layers import RNN, SimpleRNNCell, GRUCell, LSTMCell, StackedRNNCells, Dense, Embedding
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 20000
maxlen = 80
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

# Example 1: Using SimpleRNNCell inside RNN
model1 = Sequential([
    Embedding(max_features, 128),
    RNN(SimpleRNNCell(32)),
    Dense(1, activation="sigmoid")
])

# Example 2: Using GRUCell
model2 = Sequential([
    Embedding(max_features, 128),
    RNN(GRUCell(32)),
    Dense(1, activation="sigmoid")
])

# Example 3: Using LSTMCell
model3 = Sequential([
    Embedding(max_features, 128),
    RNN(LSTMCell(32)),
    Dense(1, activation="sigmoid")
])

# Example 4: Stacked cells (2 LSTM layers in one RNN)
stacked_cells = StackedRNNCells([LSTMCell(32), LSTMCell(32)])
model4 = Sequential([
    Embedding(max_features, 128),
    RNN(stacked_cells),
    Dense(1, activation="sigmoid")
])

model1
model2
model3
model4

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


<Sequential name=sequential_3, built=False>